In [1]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import OrdinalEncoder

import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv('C:/Users/Dell/Documents/book/unsuper/Projects/first/mental-heath-in-tech-2016_20161114.csv')



print('The scikit-learn version is {}.'.format(sklearn.__version__))
print('The pandas version is {}.'.format(pd.__version__))
print('The numpy version is {}.'.format(np.__version__))

# Rename column names

renamed = ["Self Employed", "Total Employees", "Tech/No Tech", "Tech Role",
           "MH Benefits", "Awareness of Company MH Care", "Formal MH Discussion",
           "MH Resources", "Anonymity", "Medical Leave", "MH Negative Impact",
           "PH Negative Impact", "Telling Coworkers", "Telling Supervisors",
           "MH PH Equal", "Observed Negative Impact", "Medical Coverage",
           "Awareness of Local Resources", "Reveal Clients (If Diagnosed)",
           "Negative Impact (If Revealed Clients)", "Reveal Employees (If Diagnosed)", 
           "Negative Impact (If Revealed Employees)", "Affected Productivity",
           "% Affected", "Previous Employers", "Prev MH Benefits", "Prev Awareness of Company MH Care",
           "Prev Formal MH Discussion", "Prev MH Resources", "Prev Anonymity",
           "Prev MH Negative Impact", "Prev PH Negative Impact", "Prev Telling Coworkers",
           "Prev Telling Supervisors", "Prev MH PH Equal", "Prev Observed Negative Impact",
           "PH Potential Employer", "PH Why/Not", "MH Potential Employer", "MH Why/Not",
           "MH Identification Career Harm", "Negative Impact Coworkers", "Sharing w/ Family/Friends",
           "Prev Observed Unsupportive Response", "Less Likely to Reveal", "MH Family History",
           "Past Mental Disorder", "Current Mental Disorder", "Condition", "Maybe Condition",
           "Professional Diagnosis", "Diagnosis", "Sought Treatment",
           "MH Interference Treatment", "MH Interference No Treatment", "Age",
           "Gender", "Country Live", "State Live", "Country", "State", "Position", "Remote"]



df.columns = renamed



get_index_self_employed = df[df['Self Employed'] == 1].index


df = df.drop(get_index_self_employed, axis = 0)


df_prep = df.copy()



# I chose to eliminate the "Country Live" and "State Live" as I am solely interested in the location where each participant is employed,
# and also removed the "why not" columns as they cannot be encoded.
df_prep.drop(['Country Live'],axis = 1,inplace=True)
df_prep.drop(['PH Why/Not'],axis = 1,inplace=True)
df_prep.drop(['MH Why/Not'],axis = 1,inplace=True)
df_prep.drop(['State Live'],axis = 1,inplace=True)



# Dealing with Gender
df['Gender'].unique()
male = ["Male", "male", "Male ", "M", "m", "man", "Cis male", "Male.",
        "Male (cis)", "Man", "Sex is male", "cis male", "Malr", "Dude",
        "I'm a man why didn't you make this a drop down question. You should of asked sex? And I would of answered yes please. Seriously how much text can this take? ",
        "mail", "M|", "male ", "Cis Male", "cisdude", "cis man", "MALE", "Male (trans, FtM)", "Male/genderqueer"]
female = ["Female", "female", "I identify as female.", "female ",
         "Female assigned at birth ", "F", "Woman", "fm", "f", "Cis female",
         "Transitioned, M2F", "Female or Multi-Gender Femme", "Female ",
         "woman", "female/woman", "Cisgender Female", "mtf", "fem",
         "Female (props for making this a freeform field, though)",
         " Female", "Cis-woman", "AFAB", "Transgender woman", "Cis female "]
other = ["Bigender", "non-binary", "Genderfluid (born female)",
        "Other/Transfeminine", "Androgynous", "male 9:1 female, roughly",
        "nb masculine", "genderqueer", "Human", "Genderfluid", "Enby",
        "genderqueer woman", "Queer", "Agender", "Fluid", "Genderflux demi-girl",
        "female-bodied; no feelings about gender", "non-binary", "Male/genderqueer",
        "Nonbinary", "Other", "none of your business", "Unicorn", "human", "Genderqueer"]




df_prep['Gender'] = df_prep['Gender'].replace(male,'Male')

df_prep['Gender'] = df_prep['Gender'].replace(female,'Female')

df_prep['Gender'] = df_prep['Gender'].replace(other,'Other')




# Removing Nan values in Gender column and replace them mode of column

mode_gender = df_prep['Gender'].mode()
df_prep['Gender'] = df_prep['Gender'].fillna(mode_gender[0])




Total = df_prep['Age'].sum()

print(Total )


# Checking if there are any NaN values in the age column and then dividing people into five groups
print(df_prep["Age"].isnull().sum())

median_age = df_prep[(df_prep['Age'] >= 18) | (df_prep['Age'] <= 75)]['Age'].median()

print("Median Age is :", median_age)

df_prep['Age'].replace(to_replace = df_prep[(df_prep['Age'] < 18) | (df_prep['Age'] > 75)]['Age'].to_list(), value = median_age, inplace = True)





def group_age(x):
    if int(x)< 20:
        return(1)
    if int(x)< 30:
        return(2)
    if int(x)< 40:
        return(3)
    if int(x)< 50:
        return(4)
    if int(x)< 60:
        return(5)
    if int(x)< 90:
        return(6)
df_prep['Age'] = df_prep['Age'].apply(group_age)




df_prep['Age'].unique()




# Dealing with total employee. 
# dividing employees into five groups

ordinal_orderings = [
    ['1-5', '6-25', '26-100', '100-500', '500-1000', 'More than 1000']
]
ordinal_features = [
    'Total Employees'
]
def ordinal_encode(df_input, columns, orderings):
    df_input = df_input.copy()
    for column, ordering in zip(columns, orderings):
        df_input[column] = df_input[column].apply(lambda x: (ordering.index(x)+1))
    return df_input
df_prep = ordinal_encode(
    df_prep, columns=ordinal_features,orderings=ordinal_orderings
)



df_prep['Total Employees'].unique()



# Dealing with position

df_prep['Position'].replace(['Back-end Developer|Front-end Developer', 'Front-end Developer|Back-end Developer'], 'Full track', inplace=True)
tech = ['Developer', 'Full Track',' DevOps', 'SysAdmin', 'Designer', 'Dev Evangelist']
df_prep['Tech Role' ] = np.where((x in df_prep['Position'] for x in tech) and (df_prep['Tech Role'].isnull()), 1.0, 0.0)
df_prep['Tech Role' ].value_counts(dropna=False)




df['MH Benefits'].unique()




# The columns in this list constitute a cluster of inquiries linked 
# to past employer. These characteristics exhibit minimal correlation with the present study.
# Consequently, I have opted to exclude said columns from the ongoing analysis.

this_list = ["Prev MH Benefits",
            "Prev Awareness of Company MH Care",
            "Prev Formal MH Discussion",
            "Prev MH Resources",
            "Prev Anonymity",
            "Prev MH Negative Impact",
            "Prev PH Negative Impact",
            "Prev Telling Coworkers",
            "Prev Telling Supervisors",
            "Prev MH PH Equal",
            "Prev Observed Negative Impact"]




df_prep = df_prep.drop(labels= this_list, axis=1)
df_prep.shape



# Dealing with nan values
# I remove columns with more than 50% of nan values
half_empty_columns = []
#go through every column 
columns = df_prep.columns.tolist()
for i in columns:
    if sum(pd.isnull(df_prep[i])) > (len(df_prep)/2):
        half_empty_columns.append(i) 



for i in half_empty_columns:
    df_prep.drop([i],  axis=1, inplace=True)



df_prep.shape



# Dividing states based on geographical location.
def create_state_encoding(state):
   west = ["Wyoming", "Nevada", "Utah", "California", "Hawaii"]
   northeast = ["Maine", "New Hampshire", "Vermont", "Massachusetts", "Rhode Island",
               "Connecticut", "New York", "Pennsylvania", "New Jersey", "Ohio"]
   east = [ "Delaware","Kentucky", "Tennessee", "North Carolina", "West Virginia", "Virginia","District of Columbia","Maryland"]
   south = ["New Mexico", "Mississippi", "Alabama", "Texas", "Arkansas", "Louisiana", "Arizona"]
   center = ["Illinois", "Indiana","South Dakota","Nebraska","Kansas", "Iowa","Colorado", "Oklahoma"]
   north = ["North Dakota", "Minnesota","Missouri","Wisconsin","Michigan"]
   north_west = ["Idaho", "Montana","Alaska","Washington","Oregon"]
   south_east = ["South Carolina","Florida", "Georgia"]
   
   if state in west:
       return 1
   elif state in northeast:
       return 2
   elif state in east:
       return 3
   elif state in south:
       return 4
   elif state in center:
       return 5
   elif state in north:
       return 6
   elif state in north_west:
       return 7
   elif state in south_east:
       return 8
   else:
       return 9 
df_prep['State'] = df_prep['State'].apply(create_state_encoding)




# As all values in 'Are you self-employed?' column are zero, I shall eliminate this column
df_prep.drop('Self Employed',  axis=1, inplace=True)



df_prep.shape




df_prep['Awareness of Company MH Care'].value_counts(dropna = False)
df_prep['Awareness of Company MH Care'].fillna('I am not sure',inplace = True)



df_prep['Formal MH Discussion'].value_counts(dropna = False)




df_prep['MH Identification Career Harm'] = df_prep['MH Identification Career Harm'].map({'Yes, I think it would': 'Yes', "No, I don't think it would":'No','Yes, it has': 'Yes', "No, it has not":'No','Maybe':'Maybe'})




df_prep['Negative Impact Coworkers'] = df_prep['Negative Impact Coworkers'].map({"No, I don't think they would": 'No','Maybe':'Maybe','Yes, they do':'Yes','Yes, I think they would':'Yes','No, they do not':'No'})



df_prep['Prev Observed Unsupportive Response'] = df_prep['Prev Observed Unsupportive Response'].map({'No': 'No','Maybe/Not sure':'Maybe','Yes, I experienced':'Yes','Yes, I observed':'Yes'})




df_prep['Prev Observed Unsupportive Response'].fillna('I am not sure',inplace = True)





ordinal_orderings = [
    ["I don't know", 'Very easy', 'Somewhat easy', 'Neither easy nor difficult','Somewhat difficult','Very difficult']
]
ordinal_features = [
    'Medical Leave'
]
def ordinal_encode_v1 (df, columns, orderings):
    df = df.copy()
    for column, ordering in zip(columns, orderings):
        df[column] = df[column].apply(lambda x: (ordering.index(x)))
    return df
df_prep = ordinal_encode_v1(
    df_prep, columns=ordinal_features,orderings=ordinal_orderings
)





def create_country(country):
    if country == "United States of America":
        return 'USA'
    elif country == "United Kingdom":
        return 'UK'
    elif country == "Canada":
        return 'Canada'
    elif country == "Germany":
        return "Germany"
    elif country == "Netherlands":
        return "Netherlands"
    elif country == "Australia":
        return "Australia"
    else:
        return "Other"
df_prep['Country'] = df_prep['Country'].apply(create_country)

df_prep.isnull().sum()


df_prep.to_csv('C:/Users/Dell/Documents/book/unsuper/send for github/clean_data_xlstat.csv')




The scikit-learn version is 1.4.2.
The pandas version is 2.0.3.
The numpy version is 1.24.4.
38569
0
Median Age is : 32.0
